<a href="https://colab.research.google.com/github/CP2J/cp2j/blob/ACJ-11-opt1-RecSys_by_DL/RecSys_with_DL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 딥러닝을 사용한 추천 시스템
* User, Item을 embedding해서 입력으로 넣으면 사용자와 아이템 간의 특성을 학습해서 추천(예측)에 이용할 수 있다.


  * < Reference : Python을 이용한 개인화 추천 시스템 2판 >

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np

# csv 파일에서 불러오기
r_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv('/content/drive/MyDrive/ml-100k/u.data', names=r_cols,  sep='\t',encoding='latin-1')
ratings = ratings[['user_id', 'movie_id', 'rating']].astype(int)            # timestamp 제거, 정수형 변환

In [ ]:
# Train-test 분리 (이렇게 나누는 방법도 있군)
from sklearn.utils import shuffle
TRAIN_SIZE = 0.75                         # 0.75 : 0.25
ratings = shuffle(ratings)                # row를 섞는다.
cutoff = int(TRAIN_SIZE * len(ratings))   # 끊는 지점
ratings_train = ratings.iloc[:cutoff]
ratings_test = ratings.iloc[cutoff:]

In [ ]:
##### (1)
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Dot, Add, Flatten
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import SGD, Adamax

In [ ]:
# Variable 초기화
K = 200                               # Latent factor(잠재요인) 수
mu = ratings_train.rating.mean()      # 전체 평균
M = ratings.user_id.max() +1          # Number of users, 사용자 아이디 최대값 : 추후 Embedding에 사용
N = ratings.movie_id.max() +1         # Number of movies

# RMSE 정의
def RMSE(y_true, y_pred):
    return tf.sqrt(tf.reduce_mean(tf.square(y_true - y_pred)))

<img src="https://drive.google.com/uc?id=1sdPLEYtb9eJ50DwhAS-yDjlYRW7eHraf" alt="딥러닝 신경망 모델" height="300" width="400">

In [ ]:
##### (2)
# Keras model
user = Input(shape=(1, ))                                             # 사용자 데이터 입력 형식 지정
item = Input(shape=(1, ))
P_embedding = Embedding(M, K, embeddings_regularizer=l2())(user)      # 사용자 Embedding layer 지정 (M, 1, K) : (M X K)의 연결을 가진다.
Q_embedding = Embedding(N, K, embeddings_regularizer=l2())(item)      # 아이템 Embedding layer 지정 (N, 1, K) : (N X K)의 연결을 가진다.
user_bias = Embedding(M, 1, embeddings_regularizer=l2())(user)        # User bias term (M, 1, ) : (M X 1)의 연결
item_bias = Embedding(N, 1, embeddings_regularizer=l2())(item)        # Item bias term (N, 1, ) : (N X 1)의 연결

# Concatenate layers
from tensorflow.keras.layers import Dense, Concatenate, Activation    # 나중에 layer 구성에서 사용할 메소드 로드
P_embedding = Flatten()(P_embedding)
Q_embedding = Flatten()(Q_embedding)
user_bias = Flatten()(user_bias)
item_bias = Flatten()(item_bias)
# Concatenate() 위해서 1차원으로 줄인다.
R = Concatenate()([P_embedding, Q_embedding, user_bias, item_bias])
# P, Q, 사용자 bias, 아이템 bias를 붙여서 하나의 layer를 만든다.

# Neural network
R = Dense(2048)(R)
R = Activation('relu')(R)
R = Dense(256)(R)
R = A